In [20]:
import mysql.connector

connection_mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="wp-ecommerce"
)

In [2]:
def get_product_name_from_id(connection_mydb,product_id):
    
    cursor = connection_mydb.cursor(dictionary=True)
    sql = "SELECT post_title FROM wp_posts WHERE ID=(%s)"
    id = (product_id,)
    
    cursor.execute(sql,id)
    
    results = cursor.fetchall()
    if  len(results) > 0:
        return results[0]['post_title']
    
    return "Unknown Product"

get_product_name_from_id(connection_mydb, 55945)

"بنطال قماش رجالي-أزرق-Levi's"

In [3]:
get_product_name_from_id(connection_mydb, 99999)

'Unknown Product'

In [4]:
import pandas as pd

def build_dataframe_associated_products(connection_mydb):
    
    df = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9])
    
    cursor = connection_mydb.cursor(dictionary=True)
    
    sql = "SELECT * FROM wp_wc_order_stats order by order_id "
    cursor.execute(sql)
    results_orders = cursor.fetchall()
    
    for order in results_orders:
        order_id = order['order_id']
        sql = "SELECT * FROM wp_wc_order_product_lookup where order_id=(%s)"
        id = (order_id,)
        cursor.execute(sql, id)
        
        results_products = cursor.fetchall()
        
        products_ids = []
        for product in results_products:
            product_id = product['product_id']
            if product_id > 0:
                products_ids.append(product_id)
        if len(products_ids) > 1:
            df = pd.concat([df, pd.DataFrame([products_ids])], ignore_index=True)
    return df

df = build_dataframe_associated_products(connection_mydb)
df

,0,1,2,3,4,5,6,7,8,9
0,55959,55979,56265,55968,NaN,NaN,NaN,NaN,NaN,NaN
1,55945,55947,56108,56261,55973,NaN,NaN,NaN,NaN,NaN
2,55890,55931,56064,56297,55898,NaN,NaN,NaN,NaN,NaN
3,56007,55982,56152,56286,55981,NaN,NaN,NaN,NaN,NaN
4,55884,55886,56082,56288,55879,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2540,56214,56247,56238,56235,NaN,NaN,NaN,NaN,NaN,NaN
2541,56354,56341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2542,55958,56005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2543,56007,56115,55976,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def prepare_transactions(df):
    
    df = df.T
    
    transactions = df.apply(lambda x: x.dropna().tolist())
    
    transactions_list = transactions.values.tolist()
    
    from mlxtend.preprocessing import TransactionEncoder
    
    te = TransactionEncoder()
    
    te_model = te.fit(transactions_list)
    
    rows = te_model.transform(transactions_list)
    
    df_transactions = pd.DataFrame(rows, columns=te_model.columns_)
    
    return df_transactions

df_transactions = prepare_transactions(df)
df_transactions

,55868,55869,55870,55873,55874,55876,55877,55878,55879,55881,...,56345,56347,56350,56351,56354,56355,56358,56359,56360,56363
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2541,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2542,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2543,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
def generate_association_rules(df_transactions, min_support, min_confidence):
    
    from mlxtend.frequent_patterns import apriori
    
    frequent_itemsets = apriori(df_transactions, min_support=min_support, use_colnames=True)
    
    from mlxtend.frequent_patterns import association_rules
    
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    
    rules = rules.sort_values(['confidence'], ascending=[False])
    
    return rules

rules = generate_association_rules(df_transactions, 0.004, 0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
15,(56235),(56193),0.009430,0.031434,0.004322,0.458333,14.580729,0.004026,1.788122,0.940283
28,(56209),(56238),0.012181,0.036149,0.005501,0.451613,12.492987,0.005061,1.757610,0.931299
51,(56323),(56305),0.010609,0.016896,0.004322,0.407407,24.112834,0.004143,1.658988,0.968806
4,(56179),(56238),0.017289,0.036149,0.006287,0.363636,10.059289,0.005662,1.514623,0.916433
1,(56179),(56218),0.017289,0.029862,0.006287,0.363636,12.177033,0.005771,1.524502,0.934026
20,(56195),(56238),0.015717,0.036149,0.005501,0.350000,9.682065,0.004933,1.482847,0.911035
40,(56227),(56228),0.014931,0.031041,0.004715,0.315789,10.173218,0.004252,1.416170,0.915370
8,(56188),(56228),0.017682,0.031041,0.005501,0.311111,10.022504,0.004952,1.406553,0.916429
35,(56245),(56217),0.014145,0.033399,0.004322,0.305556,9.148693,0.003850,1.391906,0.903475
42,(56232),(56228),0.015717,0.031041,0.004715,0.300000,9.664557,0.004227,1.384227,0.910845


In [7]:
rules=generate_association_rules(df_transactions, 0.004, 0.1)

rules.shape

(52, 10)

In [8]:
rules = generate_association_rules(df_transactions, 0.01, 0.1)
rules.shape

(0, 10)

In [9]:
support = 0.01
confidence = 0.1
n = 0
steps = 0

while n < 2000:
    support = support/2
    steps = steps+1
    rules = generate_association_rules(df_transactions, support, confidence)
    n = rules.shape[0]
    print(" rules= ",n , ", support= ", support, ", confidence= ", confidence)

 rules=  26 , support=  0.005 , confidence=  0.1
 rules=  184 , support=  0.0025 , confidence=  0.1
 rules=  535 , support=  0.00125 , confidence=  0.1
 rules=  2460 , support=  0.000625 , confidence=  0.1


In [10]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1686,"(55914, 56286)",(55892),0.000786,0.030255,0.000786,1.0,33.051948,0.000762,inf,0.970507
1739,"(55928, 55918)",(56283),0.000786,0.011788,0.000786,1.0,84.833333,0.000777,inf,0.988989
1773,"(55968, 56098)",(55963),0.000786,0.020432,0.000786,1.0,48.942308,0.000770,inf,0.980338
1772,"(55985, 55996)",(55961),0.000786,0.018861,0.000786,1.0,53.020833,0.000771,inf,0.981911
1770,"(55961, 55985)",(55996),0.000786,0.011002,0.000786,1.0,90.892857,0.000777,inf,0.989776
...,...,...,...,...,...,...,...,...,...,...
1523,(56317),(56333),0.007859,0.008644,0.000786,0.1,11.568182,0.000718,1.101506,0.920792
205,(56260),(55888),0.007859,0.023969,0.000786,0.1,4.172131,0.000597,1.084479,0.766337
1524,(56317),(56341),0.007859,0.012574,0.000786,0.1,7.953125,0.000687,1.097140,0.881188
1527,(56317),(56345),0.007859,0.012967,0.000786,0.1,7.712121,0.000684,1.096704,0.877228


In [11]:
def predict(rules, items, max_results= 6):
    
    preds = rules[rules['antecedents'] == items]
    
    preds = preds[['consequents','confidence']]
    
    preds.sort_values('confidence', ascending=False)
    
    return preds[:max_results]

In [12]:
preds = predict(rules, {56193})

preds

,consequents,confidence
1178,(56228),0.2000
1186,(56247),0.1625
1180,(56230),0.1500
1183,(56235),0.1375
1170,(56217),0.1250
1115,(56183),0.1125


In [13]:
product_id = 56193
preds = predict(rules, {product_id})

print("Item = ", get_product_name_from_id(connection_mydb, product_id))

print("Recommandations : ")

for _,row in preds.iterrows():
    
    for id in row['consequents']:
        
        name = get_product_name_from_id(connection_mydb, int(id))
        print(id, name)

Item =  فرن-بني-HP
Recommandations : 
56228 غسالة-أسود-LG
56247 غسالة-فضي-SONY
56230 غسالة-أسود-SONY
56235 مصفف شعر-أسود داكن-SONY
56217 غسالة-أسود-SAMSUNG
56183 براد-فضي-PANASONIC


In [14]:
def export_to_db(connection_mydb, rules):
  
  cursor = connection_mydb.cursor()
  
  sql = '''DROP TABLE IF EXISTS  custom_products_association'''
  
  cursor.execute(sql)
  
  sql = '''
  CREATE TABLE custom_products_association ( ID int(11) NOT NULL AUTO_INCREMENT,
    product_id_in int(11) NOT NULL,
    post_title_in text NOT NULL,
    product_id_out int(11) NOT NULL,
    post_title_out text NOT NULL,
    confidence double NOT NULL , PRIMARY KEY (ID))ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
  '''
  
  cursor.execute(sql)
  
  connection_mydb.commit()
  
  for row in rules.itertuples():
    
    antecedents = row.antecedents
    
    consequents = row.consequents
    
    confidence = row.confidence*100
    
    for product_in in antecedents:
      
      post_title_in = get_product_name_from_id(connection_mydb,product_in)
      
      for product_out in consequents:
        
        post_title_out = get_product_name_from_id(connection_mydb,product_out)
        
        
        sql='''DELETE FROM custom_products_association where product_id_in=(%s) and
        product_id_out=(%s) and confidence <= (%s) '''
        
        params = (product_in,product_out,confidence )
        
        cursor.execute(sql,params)
        
        sql = '''SELECT * FROM custom_products_association where product_id_in=(%s) and
        product_id_out=(%s) and confidence >= (%s) '''
        
        params = (product_in,product_out,confidence )
        
        cursor.execute(sql,params)
        results = cursor.fetchall()
        
        must_add = True
        if results != None and len(results) > 0:
            must_add = False
        if must_add:
            sql = "INSERT INTO custom_products_association (product_id_in, post_title_in,product_id_out,post_title_out,confidence) VALUES (%s, %s,%s,%s,%s)"
            
            val = (product_in, post_title_in, product_out, post_title_out, confidence)
          
            cursor.execute(sql, val)
          
            connection_mydb.commit()

In [15]:
def start_generate_association_rules():

    connection_mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="wp-ecommerce"
    )

    min_support = 0.001

    min_confidence = 0.001

    df = build_dataframe_associated_products(connection_mydb)

    transactions_df = prepare_transactions(df)

    rules = generate_association_rules(transactions_df,min_support ,min_confidence)

    export_to_db(connection_mydb, rules)

In [16]:
start_generate_association_rules()

In [21]:
def get_recommandation_products_ids(connection_mydb, product_id):
    
    products_recommandations = pd.DataFrame(columns=['product_id', 'post_title', 'confidence'])
    
    cursor = connection_mydb.cursor(dictionary=True)
    
    sql = "SELECT * FROM custom_products_association WHERE product_id_in=(%s) order by confidence DESC"
    
    param = (product_id, )
    
    cursor.execute(sql, param)
    
    results = cursor.fetchall()
    
    for product in results:
        
        obj = {
            'product_id':product['product_id_out'],
            'post_title':product['post_title_out'],
            'confidence': round(product['confidence'], 2)
        }
        
        products_recommandations = pd.concat([products_recommandations, pd.DataFrame([obj])], ignore_index=True)
    
    return products_recommandations

product_id = 56306

product_name = get_product_name_from_id(connection_mydb, product_id)
print("Recommendations for ",product_id, product_name)

products_recommandations = get_recommandation_products_ids(connection_mydb, product_id)
products_recommandations

Recommendations for  56306 مجموعة العناية بالبشرة-BLVD 13


,product_id,post_title,confidence
0,56350,شامبو مغزي للشعر-Boho Bunny,75.00
1,56358,زيت تصفيف للشعر-URBN Glow,60.00
2,56323,ميكاب-Wild Child,22.22
3,56347,ميكاب-SHADE,14.81
4,56303,زيت تصفيف للشعر-NOIR,14.81
5,56317,ميكاب-Luna & Clover,11.11
6,56340,ميكاب-Allora,11.11
7,56305,زيت تصفيف للشعر-BLVD 13,11.11
